In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


In [ ]:
dir_SLAND  = '/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_ctrs   = '/Data/data_ancillary/info_countries/'
dir_grids  = '/Data/grids/'
dir_forest = '/Data/forest_masks/'
dir_LSM    = '/Data/LSMs_Trendy-v8/'
dir_tmp    = '/Data/tmp/'
dir_out    = '/Data/SLAND_Trendy-v10_S2_countries/'


## Calculate SLAND in forests (using different definitions of forests, and different thresholds for forest cover)

In [ ]:
#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']#'CLASSIC-N', 'ORCHIDEE'

#Define data sets that should be used
data_sets = ['DGVMs-Forests-PFTs']#'DGVMs-NaturalLand-PFTs']#, 

#Select year for forst mask (2000, 2013, or 2016)
year = '2013'

#Select all forest or non-intact forest
selections = ['non-intact-forests']#, 'intact-forests', 'all-forests']

#Read ISO codes for countries and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
data_ctrs_ISO  = xr.open_dataset(fname_ctrs_ISO)

#Read forest mask
fname_mask   = dir_forest + 'Hansen2010_IFL_' + year + '.nc'
mask_Potapov = xr.open_dataset(fname_mask)

#Read forest fraction in 2013
fname_Hansen = dir_forest + 'ForestFraction_0.5deg_2013_regridded-ForestMask.nc'
data_Hansen  = xr.open_dataset(fname_Hansen)

#Re-index data
check_lat1 = np.max(np.abs(data_Hansen['lat'].values - mask_Potapov.lat.values))
check_lon1 = np.max(np.abs(data_Hansen['lon'].values - mask_Potapov.lon.values))
if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Coordinates do not agree')
data_Hansen = data_Hansen.reindex({'lat': mask_Potapov['lat'], 'lon': mask_Potapov['lon']}, method='nearest')

#Create NetCDF of global land area
fname_landarea_ISO = dir_ctrs + 'wrld_land-area_BLUE_TN_upd.nc'
if os.path.exists(fname_landarea_ISO): os.remove(fname_landarea_ISO)
data_landarea_ISO = 1 * (data_ctrs_ISO.ISOcode>0)
data_landarea_ISO = data_landarea_ISO.to_dataset(name='land_fraction')
data_landarea_ISO.to_netcdf(fname_landarea_ISO)        

#Loop over datasets
for data_set in data_sets:
    
    #Define different forest cover thresholds according to Hansen et al. (2013)
    if data_set=='DGVMs-Forests-PFTs':         Hansen_treshs = [0.30]#[0.05, 0.10, 0.15, 0.20, 0.25]
    elif data_set=='DGVMs-NaturalLand-PFTs':   Hansen_treshs = [0.15]    

    #Select maximum weighting factor of 50
    if 'NaturalLand' in data_set:
        for_varname = 'w_NLCF'
    else:
        for_varname = 'w_FF'
    
    #Loop over different forest cover thresholds according to Hansen et al. (2013)
    for Han2013_thresh in Hansen_treshs:
    
        #Loop over different forest definitions
        for selection in selections:

            #Select which forests to include (see Grassi et al. (2021), who used 20% threshold for forest cover)
            if selection=='non-intact-forests':
                mask_forest = data_Hansen.forest_fraction.where(mask_Potapov.Band1!=2)
                mask_forest = mask_forest > 0.0
            elif selection=='intact-forests':
                mask_forest = data_Hansen.forest_fraction.where(mask_Potapov.Band1==2)
                mask_forest = mask_forest > 0.0
            elif selection=='all-forests':
                mask_forest = data_Hansen.forest_fraction > 0.0

            #Create dicts for storing data
            SLAND_coll = dict()

            #Loop over models
            for model in models:

                print(model)

                #Conservatively regrid global land area to DGVM grid
                fname_landarea_ISO_regr = dir_tmp + 'land-area_cntrs_BLUE_TN_upd_regrid_' + model + '_for_tmp.nc'
                if os.path.exists(fname_landarea_ISO_regr): os.remove(fname_landarea_ISO_regr)
                file_grid = dir_grids + 'grid_xy_' + model
                os.system('cdo remapcon,' + file_grid + ' ' + fname_landarea_ISO + ' ' + fname_landarea_ISO_regr)

                #Read regridded land area file 
                data_landarea_ISO = xr.open_dataset(fname_landarea_ISO_regr)

                #Get file name for SLAND
                fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and ('SLAND_NBP.nc' in file) and ('NBPPFT' not in file)]
                if len(fnames)!=1:  sys.exit('Filename not unique')

                #Read SLAND data
                fname = dir_SLAND + fnames[0]
                data_SLAND = xr.open_dataset(fname)

                #Define file name for weighting factors
                if data_set=='DGVMs-Forests-PFTs':        fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_grid_' + model + '.nc'
                elif data_set=='DGVMs-NaturalLand-PFTs':  fname_weight = dir_forest + 'Weights_NaturalLandCoverFraction_DGVMs-S2-S3_grid_' + model + '.nc'

                #Distinguish whethera weighing mask exists or not for each model
                if os.path.exists(fname_weight):

                    #Flag for deleting files
                    del_files = 1

                    #Define file names for remapping (in case different Trendy versions use different grids)
                    fname_weight_tmp    = dir_tmp + 'weights_' + model + '_for_tmp.nc'
                    fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_for_tmp.nc'
                    if os.path.exists(fname_weight_tmp):     os.remove(fname_weight_tmp)
                    if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

                    #Create temporary reference grid and set grid of weighting factor
                    fname_grid_tmp = dir_tmp + 'grid_xy_for_' + model 
                    if os.path.exists(fname_grid_tmp): os.remove(fname_grid_tmp)
                    os.system('cdo -s griddes -selvar,SLAND ' + fname + ' > ' + fname_grid_tmp)
                    os.system('cdo setgrid,' + fname_grid_tmp + ' ' + fname_weight + ' ' + fname_weight_tmp)

                    #Remap weighting factors to DGVM grid 
                    os.system('cdo remapcon,' + file_grid + ' ' + fname_weight_tmp + ' ' + fname_weight_regrid)

                    #Read regridded weighting factors
                    for_weight = xr.open_dataset(fname_weight_regrid)
                    weight     = for_weight[for_varname]

                else:

                    #Flag for deleting files
                    del_files = 2

                    #Read weighting factors of all other DGVMs and calculate multi-model mean
                    if data_set=='DGVMs-Forests-PFTs':        fname_weight2 = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'  
                    elif data_set=='DGVMs-NaturalLand-PFTs':  fname_weight2 = dir_forest + 'Weights_NaturalLandCoverFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
                    for_weight = xr.open_dataset(fname_weight2)
                    for_weight = for_weight.mean('model')

                    #Define file names for remapping
                    fname_weight_tmp1   = dir_tmp + 'weights_' + model + '_for_tmp1.nc'
                    fname_weight_tmp2   = dir_tmp + 'weights_' + model + '_for_tmp2.nc'
                    fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_for_tmp.nc'
                    if os.path.exists(fname_weight_tmp1):     os.remove(fname_weight_tmp1)
                    if os.path.exists(fname_weight_tmp2):     os.remove(fname_weight_tmp2)
                    if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

                    #Save multi-model mean as NetCDF and set correct grid
                    for_weight.to_netcdf(fname_weight_tmp1)
                    file_grid_FM = '/work/mj0060/m300896/Trendy/Data/grids/grid_xy_360x720'
                    os.system('cdo setgrid,' + file_grid_FM + ' ' + fname_weight_tmp1 + ' ' + fname_weight_tmp2)

                    #Remap weighting factors to model grid
                    os.system('cdo remapcon,' + file_grid + ' ' + fname_weight_tmp2 + ' ' + fname_weight_regrid)

                    #Read regridded weighting factors
                    for_weight = xr.open_dataset(fname_weight_regrid)
                    weight     = for_weight[for_varname]

                #Get lat and lon names
                if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
                else:                              lat_name, lon_name = 'lat', 'lon'

                #Re-index data if necessary
                check_lat1 = np.max(np.abs(weight[lat_name].values - data_SLAND[lat_name].values))
                check_lon1 = np.max(np.abs(weight[lon_name].values - data_SLAND[lon_name].values))
                check_lat2 = np.max(np.abs(weight[lat_name].values - data_landarea_ISO[lat_name].values))
                check_lon2 = np.max(np.abs(weight[lon_name].values - data_landarea_ISO[lon_name].values))
                if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Coordinates do not agree')
                if check_lat2>0.001 or check_lon2>0.001:  sys.exit('Coordinates do not agree')
                weight     = weight.reindex({lat_name: data_landarea_ISO[lat_name], lon_name: data_landarea_ISO[lon_name]}, method='nearest')
                data_SLAND = data_SLAND.reindex({lat_name: data_landarea_ISO[lat_name], lon_name: data_landarea_ISO[lon_name]}, method='nearest')

                #Define temporary file name for regridding forest mask
                fname_for_tmp        = dir_tmp + 'for_selection_' + model + '_for_tmp.nc'
                fname_for_tmp_regrid = dir_tmp + 'for_selection_' + model + '_regridded_for_tmp.nc'
                if os.path.exists(fname_for_tmp):         os.remove(fname_for_tmp)
                if os.path.exists(fname_for_tmp_regrid):  os.remove(fname_for_tmp_regrid)                    

                #Calculate forest weights and save it in NetCDF
                forest_weight = 1 * (data_Hansen.forest_fraction>Han2013_thresh) & (mask_forest==1)

                #Save forest weights it in NetCDF
                forest_weight.to_dataset(name='forest').to_netcdf(fname_for_tmp)

                #Remap forest weights to DGVM grid and read regridded data
                os.system('cdo remapcon,' + file_grid + ' ' + fname_for_tmp + ' ' + fname_for_tmp_regrid)
                forest_weight_regr = xr.open_dataset(fname_for_tmp_regrid)

                #Define weight for all land and apply weighting
                weight = weight.where(weight>0, 1)
                data_SLAND = data_SLAND * weight * forest_weight_regr['forest']

                #Create dict for storing data
                SLAND_ctrs = dict()
                
                #Loop over all country codes
                for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

                    if np.mod(i, 50)==0:
                        print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

                    #Get numbeic ISO code of country
                    iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

                    #Select country in country mask
                    mask_ISO =  1 * (data_ctrs_ISO.ISOcode==iso_numeric)

                    #Define temporary file names for selecting and regridding country
                    fname_tmp      = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_for_tmp.nc'
                    fname_tmp_regr = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_regr_for_tmp.nc'
                    if os.path.exists(fname_tmp):       os.remove(fname_tmp)
                    if os.path.exists(fname_tmp_regr):  os.remove(fname_tmp_regr)

                    #Save country fraction file in NetCDF
                    mask_ISO = mask_ISO.to_dataset(name='country_fraction')
                    mask_ISO.to_netcdf(fname_tmp)

                    #Conservatively regrid country fraction to DGVM grid
                    os.system('cdo -s remapcon,' + file_grid + ' ' + fname_tmp + ' ' + fname_tmp_regr)

                    #Read regridded country fraction file
                    mask_ISO_regr = xr.open_dataset(fname_tmp_regr)

                    #Perform calculation (if country fraction is not 0 everywhere)
                    if len(mask_ISO_regr.data_vars)!=0:

                        #Create weighting factor (= fraction of country / land fraction)
                        weights = mask_ISO_regr.country_fraction / data_landarea_ISO.land_fraction

                        #Get SLAND sum in selected country
                        data_sel = (data_SLAND * weights).sum((lat_name, lon_name))

                        #Convert to Tg C/year
                        data_sel = 1000 * data_sel

                        #Save values in dict
                        SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

                    else:

                        #Set SLAND to 0 if country is too small
                        SLAND_ctrs[iso_alpha3] = np.zeros(len(data_SLAND.time))

                    #Remove temporary files
                    os.remove(fname_tmp)
                    os.remove(fname_tmp_regr)

                #Convert data to data frame (and sort by country name)
                SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
                SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)

                #Adde units in first cell
                SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')
    
                #Save data in dictionary
                SLAND_coll[model] = SLAND_ctrs_df

                #Remove temporary files
                os.remove(fname_landarea_ISO_regr)
                os.remove(fname_weight_regrid)
                os.remove(fname_for_tmp)
                os.remove(fname_for_tmp_regrid)
                if del_files==1:
                    os.remove(fname_grid_tmp)
                    os.remove(fname_weight_tmp)
                elif del_files==2:
                    os.remove(fname_weight_tmp1)
                    os.remove(fname_weight_tmp2)

            #Define output file name
            fname_out = dir_out + 'SLAND-S2-countries_' + selection + '_Weights-' + data_set + '_MaxForCover' + '{:.2f}'.format(Han2013_thresh) + '_vRemapToForestMask-Hansen2010_IFL_' + year + '_v2.xlsx'
            if os.path.exists(fname_out): os.remove(fname_out)

            #Create xlsx-file
            with pd.ExcelWriter(fname_out) as writer:

                #Loop over models
                for model in models:
                    
                    #Save as sheet in excel
                    SLAND_coll[model].to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')


## Calculate total weighted SLAND

In [ ]:
#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']#'CLASSIC-N', 'ORCHIDEE'

#Select year for forst mask (2000, 2013, or 2016)
year = '2013'

#Read ISO codes for countries and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
data_ctrs_ISO  = xr.open_dataset(fname_ctrs_ISO)

#Create NetCDF of global land area
fname_landarea_ISO = dir_ctrs + 'wrld_land-area_BLUE_TN_upd.nc'
if os.path.exists(fname_landarea_ISO): os.remove(fname_landarea_ISO)
data_landarea_ISO = 1 * (data_ctrs_ISO.ISOcode>0)
data_landarea_ISO = data_landarea_ISO.to_dataset(name='land_fraction')
data_landarea_ISO.to_netcdf(fname_landarea_ISO)

#Create dicts for storing data
SLAND_coll = dict()

#Loop over models
for model in models:

    print(model)

    #Conservatively regrid global land area to DGVM grid
    fname_landarea_ISO_regr = dir_tmp + 'land-area_cntrs_BLUE_TN_upd_regrid_' + model + '_tmp.nc'
    if os.path.exists(fname_landarea_ISO_regr): os.remove(fname_landarea_ISO_regr)
    file_grid = dir_grids + 'grid_xy_' + model
    os.system('cdo remapcon,' + file_grid + ' ' + fname_landarea_ISO + ' ' + fname_landarea_ISO_regr)

    #Read regridded land area file 
    data_landarea_ISO = xr.open_dataset(fname_landarea_ISO_regr)

    #Get file name for SLAND
    fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and ('SLAND_NBP.nc' in file) and ('NBPPFT' not in file)]
    if len(fnames)!=1:  sys.exit('Filename not unique')

    #Read SLAND data
    fname = dir_SLAND + fnames[0]
    data_SLAND = xr.open_dataset(fname)

    #Define file name for weighting factors for forest
    fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_grid_' + model + '.nc'

    #Distinguish whethera weighing mask exists or not for each model
    if os.path.exists(fname_weight):

        #Flag for deleting files
        del_files = 1

        #Define file names for remapping (in case different Trendy versions use different grids)
        fname_weight_tmp    = dir_tmp + 'weights_' + model + '_tmp.nc'
        fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_tmp.nc'
        if os.path.exists(fname_weight_tmp):     os.remove(fname_weight_tmp)
        if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

        #Create temporary reference grid and set grid of weighting factor
        fname_grid_tmp = dir_tmp + 'grid_xy_' + model 
        if os.path.exists(fname_grid_tmp): os.remove(fname_grid_tmp)
        os.system('cdo -s griddes -selvar,SLAND ' + fname + ' > ' + fname_grid_tmp)
        os.system('cdo setgrid,' + fname_grid_tmp + ' ' + fname_weight + ' ' + fname_weight_tmp)

        #Remap weighting factors to DGVM grid 
        os.system('cdo remapcon,' + file_grid + ' ' + fname_weight_tmp + ' ' + fname_weight_regrid)

        #Read regridded weighting factors
        for_weight = xr.open_dataset(fname_weight_regrid)
        weight     = for_weight.w_FF

    else:

        #Flag for deleting files
        del_files = 2

        #Read weighting factors of all other DGVMs and calculate multi-model mean
        fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
        for_weight = xr.open_dataset(fname_weight)
        for_weight = for_weight.mean('model')

        #Define file names for remapping
        fname_weight_tmp1   = dir_tmp + 'weights_' + model + '_tmp1.nc'
        fname_weight_tmp2   = dir_tmp + 'weights_' + model + '_tmp2.nc'
        fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_tmp.nc'
        if os.path.exists(fname_weight_tmp1):     os.remove(fname_weight_tmp1)
        if os.path.exists(fname_weight_tmp2):     os.remove(fname_weight_tmp2)
        if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

        #Save multi-model mean as NetCDF and set correct grid
        for_weight.to_netcdf(fname_weight_tmp1)
        file_grid_FM = '/work/mj0060/m300896/Trendy/Data/grids/grid_xy_360x720'
        os.system('cdo setgrid,' + file_grid_FM + ' ' + fname_weight_tmp1 + ' ' + fname_weight_tmp2)

        #Remap weighting factors to model grid
        os.system('cdo remapcon,' + file_grid + ' ' + fname_weight_tmp2 + ' ' + fname_weight_regrid)

        #Read regridded weighting factors
        for_weight = xr.open_dataset(fname_weight_regrid)
        weight     = for_weight.w_FF

    #Get lat and lon names
    if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
    else:                              lat_name, lon_name = 'lat', 'lon'

    #Re-index data if necessary
    check_lat1 = np.max(np.abs(weight[lat_name].values - data_SLAND[lat_name].values))
    check_lon1 = np.max(np.abs(weight[lon_name].values - data_SLAND[lon_name].values))
    check_lat2 = np.max(np.abs(weight[lat_name].values - data_landarea_ISO[lat_name].values))
    check_lon2 = np.max(np.abs(weight[lon_name].values - data_landarea_ISO[lon_name].values))
    if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Coordinates do not agree')
    if check_lat2>0.001 or check_lon2>0.001:  sys.exit('Coordinates do not agree')
    weight     = weight.reindex({lat_name: data_landarea_ISO[lat_name], lon_name: data_landarea_ISO[lon_name]}, method='nearest')
    data_SLAND = data_SLAND.reindex({lat_name: data_landarea_ISO[lat_name], lon_name: data_landarea_ISO[lon_name]}, method='nearest')

    #Define weight for all land and apply weighting
    weight = weight.where(weight>0, 1)
    data_SLAND = data_SLAND * weight
    
    #Create dicts for storing data
    SLAND_ctrs = dict()

    #Loop over all country codes
    for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

        if np.mod(i, 50)==0:
            print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

        #Get numbeic ISO code of country
        iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

        #Select country in country mask
        mask_ISO =  1 * (data_ctrs_ISO.ISOcode==iso_numeric)

        #Define temporary file names for selecting and regridding country
        fname_tmp      = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_tmp.nc'
        fname_tmp_regr = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_regr_tmp.nc'
        if os.path.exists(fname_tmp):       os.remove(fname_tmp)
        if os.path.exists(fname_tmp_regr):  os.remove(fname_tmp_regr)

        #Save country fraction file in NetCDF
        mask_ISO = mask_ISO.to_dataset(name='country_fraction')
        mask_ISO.to_netcdf(fname_tmp)

        #Conservatively regrid country fraction to DGVM grid
        os.system('cdo -s remapcon,' + file_grid + ' ' + fname_tmp + ' ' + fname_tmp_regr)

        #Read regridded country fraction file
        mask_ISO_regr = xr.open_dataset(fname_tmp_regr)

        #Perform calculation (if country fraction is not 0 everywhere)
        if len(mask_ISO_regr.data_vars)!=0:

            #Create weighting factor (= fraction of country / land fraction)
            weights = mask_ISO_regr.country_fraction / data_landarea_ISO.land_fraction

            #Get SLAND sum in selected country
            data_sel = (data_SLAND * weights).sum((lat_name, lon_name))

            #Convert to Tg C/year
            data_sel = 1000 * data_sel

            #Save values in dict
            SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

        else:

            #Set SLAND to 0 if country is too small
            SLAND_ctrs[iso_alpha3] = np.zeros(len(data_SLAND.time))

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_tmp_regr)


    #Convert data to data frame (and sort by country name)
    SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
    SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)

    #Add units in first cell
    SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')
    
    #Save data in dictionary
    SLAND_coll[model] = SLAND_ctrs_df

    #Remove temporary files
    os.remove(fname_landarea_ISO_regr)
    os.remove(fname_weight_regrid)
    if del_files==1:
        os.remove(fname_grid_tmp)
        os.remove(fname_weight_tmp)
    elif del_files==2:
        os.remove(fname_weight_tmp1)
        os.remove(fname_weight_tmp2)

#Define output file name
fname_out = dir_out + 'SLAND-S2-countries_Weights-DGVMs-Forests-PFTs_total-SLAND_vRemapToForestMask-Hansen2010_IFL_' + year + '_v2.xlsx'
if os.path.exists(fname_out): os.remove(fname_out)

#Create xlsx-file
with pd.ExcelWriter(fname_out) as writer:        
    
    #Loop over models
    for model in models:
        
        #Save as sheet in excel
        SLAND_coll[model].to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')
